In [5]:
from syserol.dataImport import importLuminex, importGlycan, createCube, getAxes, importFunction, importIGG, load_file
import pandas as pd
import numpy as np
import plotly.express as px
import statsmodels.api as sm
from functools import reduce 
from scipy.stats import zscore
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_predict

### Logistic Regression to classify subject group pairs  

In [27]:
#Import Luminex, Luminex-IGG, Subject group pairs, and Glycan into DF
df = importLuminex()
lum = df.pivot(index='subject', columns = 'variable', values = 'value')
glycan, df2 = importGlycan()
glyc = df2.pivot(index='subject', columns = 'variable', values = 'value')
subj = load_file('meta-subjects')
igg = importIGG()
igg = igg.pivot(index='subject', columns = 'variable', values = 'value')
data_frames = [lum, glyc, subj, igg]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['subject'],
                                                how='inner'), data_frames)
df_merged = df_merged.dropna()

#Subset, Z score
df_class = df_merged[["class.cp", "class.nv"]]
df_variables = df_merged.drop(['subject','class.etuv', 'class.cp', 'class.nv'], axis = 1)
df_variables = df_variables.apply(zscore) #Logistic Regression doesn't have a normalize part?


#Predict Controller vs. Progressor
Y = df_class['class.cp']
Y = (Y == 'controller').astype(int) #controllers are 1s, progressors are 0s 
X = df_variables
Y_pred = np.empty(Y.shape)

Y_pred = cross_val_predict(LogisticRegressionCV(dual=True), X, Y, cv=len(Y))


ValueError: n_splits=98 cannot be greater than the number of members in each class.

In [8]:
load_file('meta-subjects')

,subject,class.etuv,class.cp,class.nv
0,753551,EC,controller,nonviremic
1,543444,EC,controller,nonviremic
2,669831,EC,controller,nonviremic
3,782630,EC,controller,nonviremic
4,724679,EC,controller,nonviremic
...,...,...,...,...
176,477889,VC,controller,viremic
177,367794,VC,controller,viremic
178,190469,VC,controller,viremic
179,206690,VC,controller,viremic
